In [ ]:
!pip install datasets transformers seqeval razdel accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=0e29bcd94747ee7f69a8b3634084321f9d5104de9f8881852e48f6254e5e2990
  St

### Подготовка данных

In [ ]:
import pandas as pd
import csv

In [ ]:
dev_df = pd.read_csv('dev.tsv', sep='\t+', skip_blank_lines=False)
train_df = pd.read_csv('train.tsv', sep='\t+', skip_blank_lines=False)
train_df

<ipython-input-3-c3800ba8c510>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dev_df = pd.read_csv('dev.tsv', sep='\t+', skip_blank_lines=False)
<ipython-input-3-c3800ba8c510>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv('train.tsv', sep='\t+', skip_blank_lines=False)


,also,O
0,",",O
1,i,O
2,have,O
3,recently,O
4,discovered,O
...,...,...
63402,superior,B-Predicate
63403,to,O
63404,google,B-Object
63405,.,O


In [ ]:
train_df.rename(columns={'also':'tokens', 'O':'tags'}, inplace=True)
dev_df.rename(columns={'meanwhile':'tokens', 'O':'tags'}, inplace=True)
train_df.fillna('X99', inplace=True)
dev_df.fillna('X99', inplace=True)
train_df

,tokens,tags
0,",",O
1,i,O
2,have,O
3,recently,O
4,discovered,O
...,...,...
63402,superior,B-Predicate
63403,to,O
63404,google,B-Object
63405,.,O


In [ ]:
def make_sentence(name_df):
  sentence = []
  tags = []
  sentence_list = []
  tags_list = []
  count = 0
  for i in range(name_df.shape[0]):
    if name_df.iloc[i]['tokens'] != 'X99':
      sentence.append(name_df.iloc[i]['tokens'])
      tags.append(name_df.iloc[i]['tags'])
    else:
      sentence_list.append(sentence)
      tags_list.append(tags)
      sentence = []
      tags = []
  return pd.DataFrame({'tokens': sentence_list, 'tags': tags_list})

In [ ]:
train_df = make_sentence(train_df)
dev_df = make_sentence(dev_df)

In [ ]:
from datasets import Dataset, DatasetDict, load_metric

raw_datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'valid': Dataset.from_pandas(dev_df)
})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 2334
    })
    valid: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 283
    })
})

In [ ]:
labels_list = ['O', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate']
labels_list

['O',
 'B-Object',
 'I-Object',
 'B-Aspect',
 'I-Aspect',
 'B-Predicate',
 'I-Predicate']

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [labels_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(raw_datasets['train'][22:23])

{'input_ids': [[101, 1045, 2064, 1005, 1056, 3305, 2339, 2027, 1005, 1040, 2735, 2091, 1996, 10210, 3992, 2040, 2003, 2763, 2172, 25670, 2059, 1996, 5765, 4623, 2350, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 5, 0, 0, 1, 3, 4, 0, -100]]}

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)
tokenized_datasets

Map:   0%|          | 0/2334 [00:00<?, ? examples/s]

Map:   0%|          | 0/283 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2334
    })
    valid: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 283
    })
})

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=len(labels_list))
model.config.id2label = dict(enumerate(labels_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 5

args = TrainingArguments(
    'ner',
    evaluation_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs = 6,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric('seqeval')

<ipython-input-16-1d148ab27cb7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')


### Fine-tuning

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(labels_list[p])
                true_labels[-1].append(labels_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.903183102607727,
 'eval_precision': 0.026758816232500444,
 'eval_recall': 0.10603932584269662,
 'eval_f1': 0.04273383330974954,
 'eval_accuracy': 0.168859124611337,
 'eval_runtime': 5.1054,
 'eval_samples_per_second': 55.431,
 'eval_steps_per_second': 11.165}

In [ ]:
# заморозка весов
for param in model.bert.parameters():
    param.requires_grad = False

In [ ]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.889137,0.511765,0.061096,0.109159,0.807821
2,1.335500,0.626249,0.684647,0.115871,0.198198,0.817747
3,0.718700,0.553857,0.751790,0.221208,0.341834,0.833054
4,0.569100,0.524036,0.749057,0.278792,0.406346,0.840708
5,0.519600,0.510481,0.752577,0.307584,0.436690,0.845252
6,0.489500,0.506264,0.757877,0.320927,0.450913,0.847285


TrainOutput(global_step=2802, training_loss=0.7011326116634045, metrics={'train_runtime': 72.8551, 'train_samples_per_second': 192.217, 'train_steps_per_second': 38.46, 'total_flos': 329378429953452.0, 'train_loss': 0.7011326116634045, 'epoch': 6.0})

In [ ]:
# разморозка весов
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.298889,0.707857,0.695927,0.701841,0.905405
2,0.247200,0.286092,0.738346,0.689607,0.713145,0.907438
3,0.153400,0.284847,0.717857,0.705758,0.711756,0.908276


TrainOutput(global_step=1401, training_loss=0.1816446466330202, metrics={'train_runtime': 128.5719, 'train_samples_per_second': 54.46, 'train_steps_per_second': 10.897, 'total_flos': 164046661289544.0, 'train_loss': 0.1816446466330202, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [ ]:
text = ' '.join(dev_df['tokens'][0])

print(text)
print(pipe(text))

, though windows 8 is significantly at greater risk ( 1 . 73 percent ) compared to windows 8 . 1 , according to redmond ' s report , it ' s still significantly safer than windows 7 , windows xp , or windows vista .
[{'entity_group': 'Object', 'score': 0.8748944, 'word': 'windows', 'start': 9, 'end': 16}, {'entity_group': 'Predicate', 'score': 0.9703425, 'word': 'greater', 'start': 39, 'end': 46}, {'entity_group': 'Aspect', 'score': 0.7992739, 'word': 'risk', 'start': 47, 'end': 51}, {'entity_group': 'Object', 'score': 0.79966444, 'word': 'windows', 'start': 83, 'end': 90}, {'entity_group': 'Predicate', 'score': 0.984705, 'word': 'safer', 'start': 160, 'end': 165}, {'entity_group': 'Object', 'score': 0.747419, 'word': 'windows', 'start': 171, 'end': 178}]


In [ ]:
import torch

text = ' '.join(dev_df['tokens'][0])

print(text)

tokens = tokenizer(text, return_tensors='pt').to(model.device)

with torch.no_grad():
    pred = model(**tokens)
pred

, though windows 8 is significantly at greater risk ( 1 . 73 percent ) compared to windows 8 . 1 , according to redmond ' s report , it ' s still significantly safer than windows 7 , windows xp , or windows vista .


TokenClassifierOutput(loss=None, logits=tensor([[[ 0.2232,  0.2044, -0.2773, -0.3816,  0.2417, -0.4246,  0.2512],
         [ 0.2753,  0.1187, -0.2739, -0.5087,  0.0057, -0.2228,  0.3905],
         [-0.0553,  0.1443, -0.2455, -0.0409,  0.0096, -0.4532,  0.1126],
         [ 0.0156,  0.5225, -0.0627, -0.0291, -0.3075, -0.8269,  0.1283],
         [-0.1135,  0.1656, -0.3357,  0.0483, -0.0753, -0.7304,  0.3215],
         [ 0.1936,  0.6820, -0.5275, -0.0997,  0.6268, -0.0345,  0.2446],
         [ 0.0802,  0.6059, -0.0413, -0.4275,  0.4476,  0.1644,  0.2640],
         [ 0.4077,  0.6469, -0.1230, -0.1714,  0.1232, -0.1719,  0.1334],
         [ 0.4666,  0.3352, -0.2674, -0.4131,  0.2183, -0.1151, -0.0860],
         [ 0.5362, -0.1281, -0.1764,  0.2235,  0.6140, -0.1337,  0.3865],
         [ 0.2222,  0.1996, -0.2413, -0.3597,  0.0843,  0.2099, -0.0063],
         [ 0.0831,  0.4038,  0.0027, -0.5480, -0.0373,  0.2144, -0.3412],
         [ 0.0845,  0.6491,  0.4615, -0.4697,  0.4555,  0.1049, -0.0406]